In [2]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import os

In [3]:
import torch
from transformers import BertForQuestionAnswering, BertTokenizer
import pandas as pd
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# Initialize model and tokenizer
model_name = "deepset/deberta-v3-large-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Move model to CUDA if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(f"Using device: {device}")

def find_answer_text(question, text):
    try:
        # Tokenize input and move to same device as model
        inputs = tokenizer.encode_plus(question, text, return_tensors='pt', truncation=True, max_length=512)
        input_ids = inputs['input_ids'].to(device)
        token_type_ids = inputs['token_type_ids'].to(device)
        
        # Get model predictions
        with torch.no_grad():
            output = model(input_ids, token_type_ids=token_type_ids)
        
        # Get answer span
        answer_start = torch.argmax(output.start_logits)
        answer_end = torch.argmax(output.end_logits) + 1
        
        # Convert tokens to answer string
        answer = tokenizer.convert_tokens_to_string(
            tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end])
        )
        
        return answer.capitalize()
    
    except Exception as e:
        print(f"Error processing: {str(e)}")
        return None

Using device: cuda


In [4]:
question = 'What are the problems the authors are solving?'
question = 'What solution is presented?'
question = 'What problem is presented?'
question = 'What research problem is adressed?'
# question = 'What are the limitations of the solution?'
text = '''KOMPSAT-3 is a high performance remote sensing satellite, which provides 0.7 m GSD
panchromatic image and 2.8 m GSD multi-spectral image data for various applications.
KOMPSAT-3 was launched into a sun synchronous low Earth orbit on the 18th of May, 2012
and the life time of more than 7 years is expected.'''
text2 = '''Forecasting high-impact research topics
via machine learning on evolving knowledge graphs
Xuemei Gu1, ∗ and Mario Krenn1, †
1Max Planck Institute for the Science of Light, Staudtstrasse 2, 91058 Erlangen, Germany
The exponential growth in scientific publications poses a severe challenge for human researchers.
It forces attention to more narrow sub-fields, which makes it challenging to discover new impactful
research ideas and collaborations outside one’s own field. While there are ways to predict a scientific
paper’s future citation counts, they need the research to be finished and the paper written, usually
assessing impact long after the idea was conceived. Here we show how to predict the impact of
onsets of ideas that have never been published by researchers. For that, we developed a large
evolving knowledge graph built from more than 21 million scientific papers. It combines a semantic
network created from the content of the papers and an impact network created from the historic
citations of papers. Using machine learning, we can predict the dynamic of the evolving network
into the future with high accuracy (AUC values beyond 0.9 for most experiments), and thereby the
impact of new research directions. We envision that the ability to predict the impact of new ideas
will be a crucial component of future artificial muses that can inspire new impactful and interesting
scientific ideas.
'''

answer = find_answer_text(question,text)
answer

'[cls]'

In [5]:
import pandas as pd
import json

file_path = "/kaggle/input/arxiv-metadata-oai-snapshot.json"

# Load JSON data line by line (since it's likely in JSON Lines format)
data = []
with open(file_path, "r") as f:
    for line in f:
        data.append(json.loads(line))

# Convert to DataFrame
df = pd.DataFrame(data)

# Display basic info
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2725401 entries, 0 to 2725400
Data columns (total 14 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   id              object
 1   submitter       object
 2   authors         object
 3   title           object
 4   comments        object
 5   journal-ref     object
 6   doi             object
 7   report-no       object
 8   categories      object
 9   license         object
 10  abstract        object
 11  versions        object
 12  update_date     object
 13  authors_parsed  object
dtypes: object(14)
memory usage: 291.1+ MB


,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"
2,0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]"
3,0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]"
4,0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]"


In [6]:
df.head(2)

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"


In [7]:
import pandas as pd
import time
from transformers import pipeline
from tqdm import tqdm
# 1) draw a reproducible 1 000-row sample
df_sample = df.sample(n=20000, random_state=42).reset_index(drop=True)

# 2) start results as a full copy of that sample
results = df_sample.copy()

# 3) define your three new features & prompts
questions = {
    'solution': 'What solution and idea they presented?',
    'problem':  'What research problem is addressed?',
    'method':   'What methodology did they use?'
}

batch_size = 100
t0 = time.time()

# 4) for each new feature, fill it in batches
for feat, prompt in tqdm(questions.items()):
    # initialize the new column
    results[feat] = None
    
    for start in tqdm(range(0, len(df_sample), batch_size)):
        batch = df_sample.iloc[start:start+batch_size]
        
        # run your QA function over each abstract in the batch
        answers = batch['abstract'].apply(lambda txt: find_answer_text(prompt, txt))
        
        # write back into the matching rows of results
        results.loc[batch.index, feat] = answers
        
        print(f"[{feat}] rows {start+1}–{start+len(batch)} done")
        
elapsed = time.time() - t0
print(f"All batches complete in {elapsed:.1f}s")

# 5) save
results.to_csv('arxiv_qa_results_sample.csv', index=False)
print(results.head())


  0%|          | 1/200 [00:07<23:26,  7.07s/it]

[solution] rows 1–100 done



  1%|          | 2/200 [00:12<20:02,  6.07s/it]

[solution] rows 101–200 done



  2%|▏         | 3/200 [00:18<19:15,  5.87s/it]

...


100%|█████████▉| 199/200 [17:46<00:05,  5.39s/it]

[method] rows 19801–19900 done



100%|██████████| 3/3 [53:13<00:00, 1064.43s/it]

[method] rows 19901–20000 done
All batches complete in 3193.3s


                 id            submitter  \
0  quant-ph/0309212      Hilary Carteret   
1        1703.10276      Charles Marques   
2    hep-ph/9502323         Eckart Stein   
3      math/0309280  Eva Maria Feichtner   
4         1406.4226         Eddwi Hasdeo   

                                             authors  \
0                                 Hilary A. Carteret   
1     Charles Marques, Carlos Caminha, Vasco Furtado   
2  E. Stein, P. Gornicki, L. Mankiewicz, and A. S...   
3              Eva Maria Feichtner, Dmitry N. Kozlov   
4  Eddwi H. Hasdeo, Ahmad R. T. Nugraha, Mildred ...   

                                               title  \
0  Exact interferometers for the concurrence and ...   
1  An\'alise comparativa de pesquisas de origens ...   
2  QCD Sum Rule Calculation of Twist-4 Correction...   
3  A desingularization of real differentiable act...   
4  Breit-Wigner-Fano lineshapes in Raman spectra ...   

                                            comments  \
0  7 

In [8]:
results[:30]

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,solution,problem,method
0,quant-ph/0309212,Hilary Carteret,Hilary A. Carteret,Exact interferometers for the concurrence and ...,7 pages PDFLaTeX (format changed; older versio...,None,None,None,quant-ph,http://arxiv.org/licenses/nonexclusive-distrib...,In this paper we describe a set of circuits ...,"[{'version': 'v1', 'created': 'Mon, 29 Sep 200...",2016-05-30,"[[Carteret, Hilary A., ]]",A set of circuits that can measure the concurr...,Measure one type of three qubit entanglement,[cls] what methodology did they use?[sep] in t...
1,1703.10276,Charles Marques,"Charles Marques, Carlos Caminha, Vasco Furtado",An\'alise comparativa de pesquisas de origens ...,"11 pages, in Portuguese, 2 figures",None,None,None,cs.SI,http://arxiv.org/licenses/nonexclusive-distrib...,"In this paper, a comparative study was condu...","[{'version': 'v1', 'created': 'Thu, 30 Mar 201...",2017-03-31,"[[Marques, Charles, ], [Caminha, Carlos, ], [F...",Scale - free behavior can occur due to the eco...,Comparative study was conducted between comple...,Power laws
2,hep-ph/9502323,Eckart Stein,"E. Stein, P. Gornicki, L. Mankiewicz, and A. S...",QCD Sum Rule Calculation of Twist-4 Correction...,"18pp., 1 figure (uuencoded eps-file), LateX",Phys.Lett. B353 (1995) 107-113,10.1016/0370-2693(95)00544-U,UFTP preprint 380/1995,hep-ph,None,We calculate the twist-4 corrections to the ...,"[{'version': 'v1', 'created': 'Thu, 16 Feb 199...",2009-10-28,"[[Stein, E., ], [Gornicki, P., ], [Mankiewicz,...","Balitsky, braun and kolesnichenko based on a s...",[cls],Sum rule for a simpler current
3,math/0309280,Eva Maria Feichtner,"Eva Maria Feichtner, Dmitry N. Kozlov",A desingularization of real differentiable act...,"14 pages, 3 figures; substantial revision of S...",IMRN 2005:15 (2005) 881-898.,None,None,math.AG math.GT,None,We provide abelianizations of differentiable...,"[{'version': 'v1', 'created': 'Wed, 17 Sep 200...",2007-05-23,"[[Feichtner, Eva Maria, ], [Kozlov, Dmitry N., ]]",We suggest the term digitalization,[cls],[cls]
4,1406.4226,Eddwi Hasdeo,"Eddwi H. Hasdeo, Ahmad R. T. Nugraha, Mildred ...",Breit-Wigner-Fano lineshapes in Raman spectra ...,"15 pages, 4 figures",None,10.1103/PhysRevB.90.245140,None,cond-mat.mes-hall,http://arxiv.org/licenses/nonexclusive-distrib...,Excitation of electron-hole pairs in the vic...,"[{'version': 'v1', 'created': 'Tue, 17 Jun 201...",2015-06-22,"[[Hasdeo, Eddwi H., ], [Nugraha, Ahmad R. T., ...",[cls],Asymmetric breit-wigner-fano lineshape in the ...,The calculated breit-wigner-fano asymmetric fa...
5,2012.09315,Mahendra Dc,"Mahendra DC, Ding-Fu Shao, Vincent D.-H. Hou, ...",Observation of anti-damping spin-orbit torques...,None,None,None,None,cond-mat.mtrl-sci cond-mat.mes-hall,http://creativecommons.org/licenses/by-nc-sa/4.0/,High spin-orbit torques (SOTs) generated by ...,"[{'version': 'v1', 'created': 'Wed, 16 Dec 202...",2020-12-18,"[[DC, Mahendra, ], [Shao, Ding-Fu, ], [Hou, Vi...",The new material reported here provides a path...,Realize a practical spin channel in ultrafast ...,Magnetron-sputtered thin film mnpd3
6,1506.06384,Hans Arnold Winther,"Hans A. Winther, Fabian Schmidt, Alexandre Bar...",Modified Gravity N-body Code Comparison Project,"31 pages, 16 figures. Version to appear in MNRAS",None,10.1093/mnras/stv2253,None,astro-ph.CO gr-qc,http://arxiv.org/licenses/nonexclusive-distrib...,Self-consistent ${\it N}$-body simulations o...,"[{'version': 'v1', 'created': 'Sun, 21 Jun 201...",2015-11-04,"[[Winther, Hans A., ], [Schmidt, Fabian, ], [B...",We conclude that this limit is a very good app...,Obtain rigorous constraints on deviations from...,
7,2412.18258,Sylvie CLAEYSEN,"Caroline Ismeurt-Walmsley (IGF), Patrizia Gian...",The same but different: impact of animal facil...,None,None,None,None,q-bio.NC,http://arxiv.org/licenses/nonexclusive-distrib...,The gut-brain axis has emerged as a key 

In [9]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              20000 non-null  object
 1   submitter       19904 non-null  object
 2   authors         20000 non-null  object
 3   title           20000 non-null  object
 4   comments        14782 non-null  object
 5   journal-ref     6563 non-null   object
 6   doi             9020 non-null   object
 7   report-no       1348 non-null   object
 8   categories      20000 non-null  object
 9   license         16651 non-null  object
 10  abstract        20000 non-null  object
 11  versions        20000 non-null  object
 12  update_date     20000 non-null  object
 13  authors_parsed  20000 non-null  object
 14  solution        20000 non-null  object
 15  problem         20000 non-null  object
 16  method          20000 non-null  object
dtypes: object(17)
memory usage: 2.6+ MB


In [10]:
!pip install sentence_transformers

In [12]:

import os
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

df_processed = results.copy()

# 2. Select and clean the relevant text fields
fields = ['title', 'abstract', 'solution', 'problem', 'method']
for col in fields:
    if col in df_processed.columns:
        df_processed[col] = df_processed[col].fillna('')
    else:
        df_processed[col] = ''

# 3. Concatenate fields into a single text per row
separator = '\n\n'
df_processed['combined_text'] = df_processed[fields].agg(separator.join, axis=1)

# 4. Load a fast local embedding model (no API key needed)
# Using sentence-transformers 'all-MiniLM-L6-v2' which is small and quick
model = SentenceTransformer('all-MiniLM-L6-v2')

# 5. Compute embeddings in batches for memory efficiency
batch_size = 64
embeddings = []
texts = df_processed['combined_text'].tolist()
for start_idx in tqdm(range(0, len(texts), batch_size)):
    batch_texts = texts[start_idx:start_idx + batch_size]
    batch_embs = model.encode(batch_texts, show_progress_bar=False, convert_to_numpy=True)
    embeddings.append(batch_embs)

# 6. Stack embeddings and attach to DataFrame
import numpy as np
df_processed['embedding'] = list(np.vstack(embeddings))

# # 7. Save the resulting dataframe
# # Parquet preserves array-columns efficiently
# if not os.path.exists(os.path.dirname(output_path)):
#     os.makedirs(os.path.dirname(output_path))
# df_processed.to_parquet(output_path, index=False)

# print(f"Saved dataframe with embeddings to {output_path}")


100%|██████████| 313/313 [00:44<00:00,  7.06it/s]


In [13]:
df_processed

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,solution,problem,method,combined_text,embedding
0,quant-ph/0309212,Hilary Carteret,Hilary A. Carteret,Exact interferometers for the concurrence and ...,7 pages PDFLaTeX (format changed; older versio...,None,None,None,quant-ph,http://arxiv.org/licenses/nonexclusive-distrib...,In this paper we describe a set of circuits ...,"[{'version': 'v1', 'created': 'Mon, 29 Sep 200...",2016-05-30,"[[Carteret, Hilary A., ]]",A set of circuits that can measure the concurr...,Measure one type of three qubit entanglement,[cls] what methodology did they use?[sep] in t...,Exact interferometers for the concurrence and ...,"[-0.05129231, -0.042744745, -0.12601455, -0.02..."
1,1703.10276,Charles Marques,"Charles Marques, Carlos Caminha, Vasco Furtado",An\'alise comparativa de pesquisas de origens ...,"11 pages, in Portuguese, 2 figures",None,None,None,cs.SI,http://arxiv.org/licenses/nonexclusive-distrib...,"In this paper, a comparative study was condu...","[{'version': 'v1', 'created': 'Thu, 30 Mar 201...",2017-03-31,"[[Marques, Charles, ], [Caminha, Carlos, ], [F...",Scale - free behavior can occur due to the eco...,Comparative study was conducted between comple...,Power laws,An\'alise comparativa de pesquisas de origens ...,"[0.057466254, -0.03783397, -0.027481401, 0.015..."
2,hep-ph/9502323,Eckart Stein,"E. Stein, P. Gornicki, L. Mankiewicz, and A. S...",QCD Sum Rule Calculation of Twist-4 Correction...,"18pp., 1 figure (uuencoded eps-file), LateX",Phys.Lett. B353 (1995) 107-113,10.1016/0370-2693(95)00544-U,UFTP preprint 380/1995,hep-ph,None,We calculate the twist-4 corrections to the ...,"[{'version': 'v1', 'created': 'Thu, 16 Feb 199...",2009-10-28,"[[Stein, E., ], [Gornicki, P., ], [Mankiewicz,...","Balitsky, braun and kolesnichenko based on a s...",[cls],Sum rule for a simpler current,QCD Sum Rule Calculation of Twist-4 Correction...,"[-0.13641588, 0.0025326945, 0.06542231, 0.0233..."
3,math/0309280,Eva Maria Feichtner,"Eva Maria Feichtner, Dmitry N. Kozlov",A desingularization of real differentiable act...,"14 pages, 3 figures; substantial revision of S...",IMRN 2005:15 (2005) 881-898.,None,None,math.AG math.GT,None,We provide abelianizations of differentiable...,"[{'version': 'v1', 'created': 'Wed, 17 Sep 200...",2007-05-23,"[[Feichtner, Eva Maria, ], [Kozlov, Dmitry N., ]]",We suggest the term digitalization,[cls],[cls],A desingularization of real differentiable act...,"[-0.11120024, -0.05407834, 0.005986905, -0.039..."
4,1406.4226,Eddwi Hasdeo,"Eddwi H. Hasdeo, Ahmad R. T. Nugraha, Mildred ...",Breit-Wigner-Fano lineshapes in Raman spectra ...,"15 pages, 4 figures",None,10.1103/PhysRevB.90.245140,None,cond-mat.mes-hall,http://arxiv.org/licenses/nonexclusive-distrib...,Excitation of electron-hole pairs in the vic...,"[{'version': 'v1', 'created': 'Tue, 17 Jun 201...",2015-06-22,"[[Hasdeo, Eddwi H., ], [Nugraha, Ahmad R. T., ...",[cls],Asymmetric breit-wigner-fano lineshape in the ...,The calculated breit-wigner-fano asymmetric fa...,Breit-Wigner-Fano lineshapes in Raman spectra ...,"[-0.03055581, -0.02238005, -0.0347739, 0.02316..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,hep-th/0010199,Klaus Kirsten,"Stuart Dowker, Peter Gilkey and Klaus Kirsten",On properties of the asymptotic expansion of t...,"16 pages, LaTeX",Int.J.Math. 12 (2001) 505-518,None,None,hep-th,None,The spectral problem where the field satisfi...,"[{'version': 'v1', 'created': 'Mon, 23 Oct 200...",2007-05-23,"[[Dowker, Stuart, ], [Gilkey, Peter, ], [Kirst...",The spectral problem where the field satisfies...,The spectral problem,[cls],On properties of the asymptotic expansion of t...,"[-0.040779956, -1.0574469e-05, 0.031447142, 0...."
19996,hep-th/0008137,Sergio Zerbini,G. Cognola and S. Zerbini,On the dimensional reduced theories,"9 pages, proceeding of Londrina Conference, Ap...",None,10.1142/97898128

In [14]:
import pandas as pd

# Keep only 'embedding' and 'id' columns
df_processed = df_processed[['embedding', 'id']]

# Save to a new CSV file
df_processed.to_csv('filtered_data.csv', index=False)


In [15]:
df_processed.head()

,embedding,id
0,"[-0.05129231, -0.042744745, -0.12601455, -0.02...",quant-ph/0309212
1,"[0.057466254, -0.03783397, -0.027481401, 0.015...",1703.10276
2,"[-0.13641588, 0.0025326945, 0.06542231, 0.0233...",hep-ph/9502323
3,"[-0.11120024, -0.05407834, 0.005986905, -0.039...",math/0309280
4,"[-0.03055581, -0.02238005, -0.0347739, 0.02316...",1406.4226
